<a href="https://colab.research.google.com/github/OVP2023/E-CAP-2024/blob/main/%D0%A4%D0%BE%D1%80%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2_%D0%B8_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Формирование признаков и обучение модели

In [68]:
import pandas as pd
import numpy as np

In [83]:
train = pd.read_parquet('/content/drive/MyDrive/ecap/mydata2.parquet')
train.head(7)

,variantid1,variantid2,target,Cosine_similarity_names,art1,color1,country1,brand1,art2,color2,country2,brand2
0,1447875869,1447872068,1,1.000000,None,[светло-бежевый],[Япония],None,None,[светло-бежевый],[Япония],None
1,1176231201,284733670,1,0.989477,None,None,[Россия],[Джага Джага],None,None,[Россия],[Джага Джага]
2,658617865,549848659,0,0.966868,[83809],None,[Нидерланды],[Hill's],[10039],None,[Чехия],[Hill's]
3,719320625,719370486,1,1.000000,None,[серебристый],[Китай],[Elitech],None,[серебристый],[Китай],[Elitech]
4,1067645658,949954740,0,0.986480,None,None,None,[Россия],None,[розовый],[Россия],[Нет бренда]
5,504121134,304997246,0,0.991861,None,None,[Великобритания],[Puriferm],None,None,[Великобритания],[Puriferm]
6,690909146,1044397832,1,0.938379,[77RP008_MASTER KIT],None,None,[MasterKit],[MASTERKIT_77RP008],None,None,[MasterKit]


In [84]:
train['color1'].fillna(0, inplace=True)
train['color2'].fillna(0, inplace=True)
train['brand1'].fillna(0, inplace=True)
train['brand2'].fillna(0, inplace=True)
train['country1'].fillna(0, inplace=True)
train['country2'].fillna(0, inplace=True)


for i in range(0, train.shape[0]):
    if train['art1'][i]==train['art2'][i] and train['art1'][i]!=None and train['art2'][i]!=None:
        train.loc[i,'yesart']=1
        train.loc[i,'noart']=0
    elif train['art1'][i]!=train['art2'][i] and train['art1'][i]!=None and train['art2'][i]!=None:
        train.loc[i,'noart']=1
        train.loc[i,'yesart']=0
    else:
        train.loc[i,'yesart']=0
        train.loc[i,'noart']=0

    if type(train['color1'][i])!=int and type(train['color2'][i])!=int:
        if train['color1'][i][0] == train['color2'][i][0]:
            train.loc[i,'yescolor']=1
            train.loc[i,'nocolor']=0
        else:
            train.loc[i,'nocolor']=1
            train.loc[i,'yescolor']=0
    else:
        train.loc[i,'nocolor']=0
        train.loc[i,'yescolor']=0

    if type(train['brand1'][i])!=int and type(train['brand2'][i])!=int:
        if train['brand1'][i][0] == train['brand2'][i][0]:
            train.loc[i,'yesbrand']=1
            train.loc[i,'nobrand']=0
        else:
            train.loc[i,'nobrand']=1
            train.loc[i,'yesbrand']=0
    else:
        train.loc[i,'nobrand']=0
        train.loc[i,'yesbrand']=0

    if type(train['country1'][i])!=int and type(train['country2'][i])!=int:
        if train['country1'][i][0] == train['country2'][i][0]:
            train.loc[i,'yescountry']=1
            train.loc[i,'nocountry']=0
        else:
            train.loc[i,'nocountry']=1
            train.loc[i,'yescountry']=0
    else:
        train.loc[i,'nocountry']=0
        train.loc[i,'yescountry']=0





In [85]:
train.head(7)

,variantid1,variantid2,target,Cosine_similarity_names,art1,color1,country1,brand1,art2,color2,country2,brand2,yesart,noart,yescolor,nocolor,nobrand,yesbrand,yescountry,nocountry
0,1447875869,1447872068,1,1.000000,None,[светло-бежевый],[Япония],0,None,[светло-бежевый],[Япония],0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1176231201,284733670,1,0.989477,None,0,[Россия],[Джага Джага],None,0,[Россия],[Джага Джага],0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,658617865,549848659,0,0.966868,[83809],0,[Нидерланды],[Hill's],[10039],0,[Чехия],[Hill's],0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,719320625,719370486,1,1.000000,None,[серебристый],[Китай],[Elitech],None,[серебристый],[Китай],[Elitech],0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1067645658,949954740,0,0.986480,None,0,0,[Россия],None,[розовый],[Россия],[Нет бренда],0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,504121134,304997246,0,0.991861,None,0,[Великобритания],[Puriferm],None,0,[Великобритания],[Puriferm],0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6,690909146,1044397832,1,0.938379,[77RP008_MASTER KIT],0,0,[MasterKit],[MASTERKIT_77RP008],0,0,[MasterKit],0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [86]:
train = train.drop(columns=(['art1','art2','color1','color2','brand1','brand2','country1','country2']))

In [87]:
train.head()

,variantid1,variantid2,target,Cosine_similarity_names,yesart,noart,yescolor,nocolor,nobrand,yesbrand,yescountry,nocountry
0,1447875869,1447872068,1,1.000000,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1176231201,284733670,1,0.989477,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,658617865,549848659,0,0.966868,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,719320625,719370486,1,1.000000,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1067645658,949954740,0,0.986480,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [88]:
#метод стохастического градиентного спуска и оценка точности модели
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X=train[['Cosine_similarity_names','yesart','noart','yescolor','nocolor','yesbrand','nobrand','yescountry','nocountry']] # признаки X
y=train['target'] # целевая переменная y

#Сформируйте X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

sgd_clf = SGDClassifier(random_state=42) # классификатор на основе метода стохастического градиентного спуска (англ. Stochastic Gradient Descent SGD)
sgd_clf.fit(X_train, y_train) # обучаем классификатор
# Для расчета матрицы ошибок сначала понадобится иметь набор прогнозов, чтобы их можно было сравнивать с фактическими целями
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)
print(confusion_matrix(y_train, y_train_pred))
print(accuracy_score(y_train, y_train_pred))

[[297804 126751]
 [148376 245030]]
0.6636428876193363


1) Пока правильность всего 55% на 2 признаках (name и артикул разбиваем на 2), нужно добавить еще 4 Признаки бренда, страны-изготовителя, типа товара, цвета
2) добавлением признака цвета увеличил до 60% 3) добавлением признака бренда увеличил до 65% 4)но добавлением признака страны-изготовителя улучшил модель только на 1.4% до 66.4%